# Ergodicity Basic Simulation - Results Analysis

In [ ]:
import re

from pathlib import Path
from PIL import Image

results_dir = Path("results")
pattern = "*Growth Rate*.png"
image_paths = list(results_dir.glob(pattern))

if not image_paths:
    print(f"No PNG files found in {results_dir} matching pattern containing 'Growth Rate'.")
else:
    def natural_key(p):
        # Split into text and number chunks for natural sort
        return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', p.stem)]
    image_paths.sort(key=natural_key)

    frames = []
    for p in image_paths:
        try:
            img = Image.open(p)
            frames.append(img.convert("RGBA"))
        except Exception as e:
            print(f"Skipping {p}: {e}")

    if len(frames) <= 1:
        print("Need at least 2 frames to build an animation.")
    else:
        output_path = results_dir / "Growth Rate Animation.gif"
        frames[0].save(
            output_path,
            save_all=True,
            append_images=frames[1:],
            duration=50,  # ms per frame
            loop=0,        # 0 = infinite loop
            disposal=2
        )
        print(f"Animated GIF saved to: {output_path} ({len(frames)} frames)")

Animated GIF saved to: results\Growth Rate Animation.gif (1125 frames)


In [37]:
import pickle
import re

from pathlib import Path
from PIL import Image

from ergodic_insurance.monte_carlo import SimulationResults

results_dir = Path("results")

pkl_paths = sorted(results_dir.glob("*.pkl"))

all_configurations = {}
if not pkl_paths:
    print(f"No pickle files found in {results_dir}.")
else:
    for path in pkl_paths:
        try:
            with open(path, "rb") as f:
                all_configurations[path.stem] = pickle.load(f)
        except Exception as e:
            print(f"Skipping {path.name}: {e}")
    print(f"Loaded {len(all_configurations)} pickle files into all_configurations.")

Loaded 1250 pickle files into all_configurations.


In [38]:
def get_object_methods(obj):
    """
    Returns a list of all callable methods of a given Python object,
    excluding "dunder" (double underscore) methods.
    """
    methods = []
    for name in dir(obj):
        attribute = getattr(obj, name)
        if callable(attribute) and not name.startswith("__"):
            methods.append(name)
    return methods


one_configuration = all_configurations['Cap (5M) -    ATR (0.5) -    EBITABL (0.1) -    XS_Kurt (4611) -    Ded (0K) -    LR (0.6) -    0K Sims -    50 Yrs']

print("Methods:", get_object_methods(one_configuration))

print("Attributes:", one_configuration.__dict__.keys())

print("Growth Rate:", one_configuration.growth_rates.mean())

print("Risk of Ruin:", one_configuration.ruin_probability)

Methods: ['summary']
Attributes: dict_keys(['final_assets', 'annual_losses', 'insurance_recoveries', 'retained_losses', 'growth_rates', 'ruin_probability', 'metrics', 'convergence', 'execution_time', 'config', 'performance_metrics', 'aggregated_results', 'time_series_aggregation', 'statistical_summary', 'summary_report', 'bootstrap_confidence_intervals'])
Growth Rate: -0.04166359454393387
Risk of Ruin: {'5': 0.0, '10': 0.0, '15': 0.0, '20': 0.0, '25': 0.0, '30': 0.0, '35': 0.0, '40': 0.0, '45': 0.0, '50': 0.0}


In [48]:
one_configuration.final_assets.shape

(100,)

In [46]:
one_configuration.aggregated_results

{'count': 100,
 'mean': np.float32(622668.06),
 'std': np.float32(0.0),
 'min': np.float32(622668.06),
 'max': np.float32(622668.06),
 'percentiles': {'p1': np.float64(622668.0625),
  'p5': np.float64(622668.0625),
  'p10': np.float64(622668.0625),
  'p25': np.float64(622668.0625),
  'p50': np.float64(622668.0625),
  'p75': np.float64(622668.0625),
  'p90': np.float64(622668.0625),
  'p95': np.float64(622668.0625),
  'p99': np.float64(622668.0625)},
 'moments': {'variance': np.float32(0.0),
  'skewness': np.float32(nan),
  'kurtosis': np.float32(nan),
  'coefficient_variation': np.float32(0.0)}}

In [50]:
print(one_configuration.summary())

Simulation Results Summary
Simulations: 100
Years: 50
Execution Time: 2.28s
Ruin Probability:
  Year 5: 0.00%
  Year 10: 0.00%
  Year 15: 0.00%
  Year 20: 0.00%
  Year 25: 0.00%
  Year 30: 0.00%
  Year 35: 0.00%
  Year 40: 0.00%
  Year 45: 0.00%
  Year 50: 0.00%
Mean Final Assets: $622,668
Mean Growth Rate: -0.0417
VaR(99%): $63,070,128
TVaR(99%): $70,994,416
Convergence R-hat: 0.000

Performance Summary
Total Time: 2.28s
Setup: 0.00s
Computation: 2.28s
Serialization: 0.00s (0.0% overhead)
Reduction: 0.00s
Peak Memory: 0.2 MB
CPU Utilization: 0.0%
Throughput: 44 items/s
Speedup: 1.00x

Advanced Aggregation Results:
  p1: $622,668
  p5: $622,668
  p10: $622,668
  p25: $622,668
  p50: $622,668
  p75: $622,668
  p90: $622,668
  p95: $622,668
  p99: $622,668



In [66]:
import pickle
import re

from pathlib import Path
from PIL import Image

from ergodic_insurance.monte_carlo import SimulationResults

def _parse_number(text):
    text = text.strip().replace(",", "")
    m = re.fullmatch(r'([+-]?\d+(?:\.\d+)?)([KMB])?$', text, re.I)
    if not m:
        # fallback: plain int/float or leave as-is
        try:
            return int(text)
        except ValueError:
            try:
                return float(text)
            except ValueError:
                return text
    num = float(m.group(1))
    mult = {"K": 1_000, "M": 1_000_000, "B": 1_000_000_000}.get((m.group(2) or "").upper(), 1)
    val = num * mult
    return int(val) if val.is_integer() else val


def parse_config_key(key: str) -> dict:
    parts = re.split(r"\s*-\s*", key.strip())
    out = {}
    for part in parts:
        if not part:
            continue

        # e.g. "Cap (100M)"
        m = re.match(r"^([A-Za-z_]+)\s*\(\s*([^)]+)\s*\)$", part)
        if m:
            out[m.group(1)] = _parse_number(m.group(2))
            continue

        # e.g. "0K Sims" or "50 Yrs"
        m = re.match(r"^([+-]?\d+(?:\.\d+)?)\s*([KMB])?\s*([A-Za-z_]+)$", part)
        if m:
            value = _parse_number((m.group(1) or "") + (m.group(2) or ""))
            out[m.group(3)] = value
            continue

        # flags like "NOINS"
        if part.upper() == "NOINS":
            out["NOINS"] = True

    return out

results_dir = Path("results")

pkl_paths = sorted(results_dir.glob("*.pkl"))

all_configurations = {}
if not pkl_paths:
    print(f"No pickle files found in {results_dir}.")
else:
    for path in pkl_paths:
        try:
            with open(path, "rb") as f:
                one_config = pickle.load(f)
                growth_rate = one_config.growth_rates.mean()
                ror = one_config.ruin_probability
                all_configurations[path.stem] = {
                    "growth_rate": growth_rate,
                    "risk_of_ruin": ror
                }
        except Exception as e:
            print(f"Skipping {path.name}: {e}")
    print(f"Loaded {len(all_configurations)} pickle files into all_configurations.")

parsed_params_by_key = {k: parse_config_key(k) | all_configurations[k] for k in all_configurations.keys()}
parsed_params_by_key

Loaded 1250 pickle files into all_configurations.


{'Cap (100M) -    ATR (0.5) -    EBITABL (0.1) -    XS_Kurt (3705) -    Ded (0K) -    LR (0.6) -    0K Sims -    50 Yrs': {'Cap': 100000000,
  'ATR': 0.5,
  'EBITABL': 0.1,
  'XS_Kurt': 3705,
  'Ded': 0,
  'LR': 0.6,
  'Sims': 0,
  'Yrs': 50,
  'growth_rate': np.float64(0.01273997500538826),
  'risk_of_ruin': {'5': 0.0,
   '10': 0.0,
   '15': 0.0,
   '20': 0.0,
   '25': 0.0,
   '30': 0.0,
   '35': 0.0,
   '40': 0.0,
   '45': 0.0,
   '50': 0.0}},
 'Cap (100M) -    ATR (0.5) -    EBITABL (0.1) -    XS_Kurt (3705) -    Ded (0K) -    LR (0.7) -    0K Sims -    50 Yrs': {'Cap': 100000000,
  'ATR': 0.5,
  'EBITABL': 0.1,
  'XS_Kurt': 3705,
  'Ded': 0,
  'LR': 0.7,
  'Sims': 0,
  'Yrs': 50,
  'growth_rate': np.float64(0.01273997500538826),
  'risk_of_ruin': {'5': 0.0,
   '10': 0.0,
   '15': 0.0,
   '20': 0.0,
   '25': 0.0,
   '30': 0.0,
   '35': 0.0,
   '40': 0.0,
   '45': 0.0,
   '50': 0.0}},
 'Cap (100M) -    ATR (0.5) -    EBITABL (0.1) -    XS_Kurt (3705) -    Ded (0K) -    LR (0.8) -    